In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_datagenerator = train_datagenerator.flow_from_directory('train', target_size=(128,128), batch_size=40, class_mode='binary')
test_datagenerator = test_datagenerator.flow_from_directory('test', target_size=(128,128), batch_size=10,class_mode='binary')

Found 600 images belonging to 2 classes.
Found 66 images belonging to 2 classes.


In [3]:
import tensorflow.keras.layers as m

model = tf.keras.models.Sequential()
model.add(m.Conv2D(32, (3,3), padding='same', activation='relu',input_shape=(128,128,3)))
model.add(m.MaxPooling2D((2,2),2))

model.add(m.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(m.MaxPooling2D((2,2),2))

model.add(m.Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(m.MaxPooling2D((2,2),2))

model.add(m.Flatten())
model.add(m.Dense(512, activation='relu'))
model.add(m.Dense(1, activation='sigmoid'))

In [4]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0

In [5]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001),metrics=['accuracy'])

In [6]:
DESIRED_ACC = 0.90

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if((logs.get('acc')>DESIRED_ACC)and (logs.get('val_acc')>DESIRED_ACC)):
            print("Reached 85 %")
            self.model.stop_training = True
            
        
callbacks = myCallback()

In [8]:
model.fit_generator(train_datagenerator, epochs=50, validation_data = test_datagenerator)

Epoch 1/50
15/15 [==============================] - 13s 846ms/step - loss: 0.4156 - acc: 0.8633 - val_loss: 0.2392 - val_acc: 0.9394
Epoch 2/50
15/15 [==============================] - 8s 552ms/step - loss: 0.3249 - acc: 0.8683 - val_loss: 0.2186 - val_acc: 0.9242
Epoch 3/50
15/15 [==============================] - 8s 551ms/step - loss: 0.2124 - acc: 0.9250 - val_loss: 0.2306 - val_acc: 0.8939
Epoch 4/50
15/15 [==============================] - 8s 553ms/step - loss: 0.1773 - acc: 0.9283 - val_loss: 0.1880 - val_acc: 0.9394
Epoch 5/50
15/15 [==============================] - 8s 561ms/step - loss: 0.1099 - acc: 0.9667 - val_loss: 0.1439 - val_acc: 0.9697
Epoch 6/50
15/15 [==============================] - 9s 584ms/step - loss: 0.0967 - acc: 0.9667 - val_loss: 0.2371 - val_acc: 0.9091
Epoch 7/50
15/15 [==============================] - 8s 555ms/step - loss: 0.1008 - acc: 0.9667 - val_loss: 0.1802 - val_acc: 0.9394
Epoch 8/50
15/15 [==============================] - 8s 552ms/step - loss: 0

In [9]:
model.save('spamham.h5')